<h1>Chicago Crime Analysis - Usando Data Science para entender  crime em Chicagono ano de  2018</h1>


<strong>Bibliotecas</strong>
<ul><li>pandas</li>
    <li>matplotlib</li>
    <li>seaborn</li>
    <li>numpy</li>
    <li>datetime</li>
    <li>folium</li>
    <li>bokeh</li>
</ul>

<strong>URL do  Dataset : </strong>https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-present/ijzp-q8t2

#Principais objetivos do Projeto:
##Crimes Comuns em Chicago
*   Um gráfico que descreve os crimes mais ocorridos em Chicago em 2018.
*   Uma comparação da frequência das 5 infrações mais cometidas
##Prisões e a cidade de Chicago
*   Qual é a probabilidade de uma prisão?
*   Distribuição das prisões ao longo do mês
*   Como a prisão varia de acordo com o tipo de crime?
##Crime versus Tempo
*   Como o crime varia ao longo do ano?
*   Quais horários são mais inseguros?
*   Que crimes são mais propensos a acontecer na escuridão do que pela manhã?
*   Sua casa está protegida contra roubos durante o dia?
*   Visualize o padrão de 24 horas de um crime

##Crime vs Locais
*   Qual distrito é o mais seguro para se viver? Qual distrito é "distrito do pecado"?
*   Visualize os crimes mais ocorridos por distrito
*   Concentração do Crime na Cidade da lista
*   Você mora na área X. Você deseja ver as estatísticas de criminalidade em X no ano de 2018, especialmente de “Roubo”. Visualize um gráfico de série temporal para ver o mesmo.
*   Item Existem padrões óbvios nas atividades de gangues distintas em Chicago em 2018?









In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from datetime import datetime

<h2>Obtendo os dados</h2>
Para acessar os dados compactados, precisamoes da lib requests, que acessa a internet, e a zipfile para descompactar

In [ ]:
import requests
from io import BytesIO
from zipfile import ZipFile
import os

# URL do arquivo zip no GitHub
url = "https://github.com/Jashu-MJ/Chicago-Crime-Data-Analysis/raw/master/crimes_2018.zip"

# Fazer o download do arquivo zip
response = requests.get(url)
zip_file = ZipFile(BytesIO(response.content))

# Listar os arquivos no zip (opcional)
print("Arquivos no zip:", zip_file.namelist())

# Extrair o conteúdo do zip para um diretório temporário
temp_dir = "crimes_2018"
zip_file.extractall(temp_dir)

# Listar os arquivos extraídos (opcional)
print("Arquivos extraídos:", os.listdir(temp_dir))

# Ler o arquivo CSV (ou outro formato) em um DataFrame Pandas
csv_file_path = os.path.join(temp_dir, "crimes_2018.csv")
df = pd.read_csv(csv_file_path)

# Exibir as primeiras linhas do DataFrame
print(df.head())

<h3>Apresentando os dados....</h3>

In [ ]:
# Numero de linhas e colunas
df.shape

In [ ]:
# Resumo estatísticos dos dados: numero de vezes que aparece, media, desvio padrão, minimo, percentis e máximo
df.describe()

In [ ]:
#exibindo o nome das colunas e o tipo de variável.
df.info()

In [ ]:
# Mostrando as 5 primeiras linhas
df.head()

In [ ]:
# Exibindo o nome das colunas
print("Nomes dos dados (colunas) armazenados :\n", list(df.columns))

<h3>Listagem dos crimes em Chicago: quais os tipos de crimes que são registrados</h3>

In [ ]:
# Na coluna Primary Type, aparecem os tipos de crime. o comando unique retorna uma ocorrencia de cada, não exibindo tipos de crimes repetidos
crimes = df['Primary Type'].unique()
print("Quantos  tipos de crimes são registrados pela polícia de Chigaco:", len(crimes))
print()
print("Os crimes são: :\n", crimes)

###Vamos fazer um mapeamento, entre as palavras em Ingles e as em portugies, e alterar todos os Primary Type para portugues



In [ ]:
mapeamento_tipo = {
    'THEFT': 'Roubo',
    'CRIM SEXUAL ASSAULT': 'Assalto Sexual Criminoso',
    'BATTERY': 'Agressão',
    'DECEPTIVE PRACTICE': 'Fraude',
    'OTHER OFFENSE': 'Outra Ofensa',
    'ASSAULT': 'Assalto',
    'MOTOR VEHICLE THEFT': 'Roubo de Veículo',
    'CRIMINAL TRESPASS': 'Invasão Criminal',
    'HOMICIDE': 'Homicídio',
    'OFFENSE INVOLVING CHILDREN': 'Ofensa Envolvendo Crianças',
    'SEX OFFENSE': 'Ofensa Sexual',
    'WEAPONS VIOLATION': 'Violação de Armas',
    'ROBBERY': 'Assalto à Mão Armada',
    'BURGLARY': 'Roubo Residencial',
    'KIDNAPPING': 'Sequestro',
    'CRIMINAL DAMAGE': 'Invasão',
    'PUBLIC PEACE VIOLATION': 'Violação da Paz Pública',
    'INTIMIDATION': 'Intimidação',
    'OBSCENITY': 'Obscenidade',
    'STALKING': 'Perseguição',
    'ARSON': 'Incêndio Criminoso',
    'NARCOTICS': 'Narcóticos',
    'PROSTITUTION': 'Prostituição',
    'LIQUOR LAW VIOLATION': 'Violação da Lei de Bebidas Alcoólicas',
    'CONCEALED CARRY LICENSE VIOLATION': 'Violação da Licença de Porte Oculto',
    'INTERFERENCE WITH PUBLIC OFFICER': 'Interferência com Oficial Público',
    'HUMAN TRAFFICKING': 'Tráfico de Seres Humanos',
    'NON-CRIMINAL': 'Não Criminal',
    'PUBLIC INDECENCY': 'Indecência Pública',
    'GAMBLING': 'Jogo',
    'OTHER NARCOTIC VIOLATION': 'Outra Violação de Narcóticos',
    'NON-CRIMINAL (SUBJECT SPECIFIED)': 'Não Criminal (Especificado)',
}


# Substituir os valores na coluna Primary Type
df['Primary Type'] = df['Primary Type'].replace(mapeamento_tipo)
crimes = df['Primary Type'].unique()
print("Quantos  tipos de crimes são registrados pela polícia de Chigaco:", len(crimes))
print()
print("Os crimes são: :\n", crimes)

<h2>Um dos grandes problemas em Datasets</h2>
O que fazer quando faltam dados ???

In [ ]:
# quantos dados estão faltando  ? isna define os que são NaN
# o primeiro sum retorna um dataset com quantos NaN tem em cada coluna, e o segundo, faz o somatorio de todos
print("Número de valores que faltam no dataset : ", df.isna().sum().sum())

In [ ]:
# Vamos verificar quais valores estão faltando, para que a policia possa efetuar correções no sistema de captação de dados
missing_values = list(df.isna().sum())
# missing values é a lista de valores faltando em cada coluna

cols = list(df.columns)
col_final = []
for i in range(len(cols)):
    if (missing_values[i] == 0):
        cols[i]="Outros"
d = dict(zip(cols, missing_values)) # cria um "dicionário", que é uma estrutura de dados que tem as colunas e os valores que faltam nela

print("Número de Valores ausentes em cada uma das características (counas) >>")
missing_vals = pd.DataFrame(d, index=["Valores Ausentes"]) # criando um novo dataframe contendo os valores que faltam
missing_vals.head()

In [ ]:
# Plotting the missing values in the dataset
x = list(d.keys())
y = list(d.values())
sns.barplot(x=x, y=y, palette="GnBu_d")
plt.xticks(rotation=90)
plt.title("Valores ausentes no dataset", fontdict = {'fontsize': 20})
plt.ylabel("Número de valores ausentes por Coluna", fontdict={'fontsize': 15})
plt.show()

<h4>Um pouco sobre estes valores que faltam</h4>

<strong> Por quê faltam informações?</strong>
<p>Podemos observar que a maioria das informações que faltam no dataset original se referem a localização do crime. Estas infromações as vezes não são fornecidas adequadamente para a polícia, ou podem ter sido não anotadas pelo agente<br>
<h4> Vamos então "limpar" estes dados do dataframe. O comando DROPNA "dropa" (elimina) os dados NaN.


In [ ]:
# The simplest cleaning technique here would be to drop all the rows with atleast one missing value
df = df.dropna()
df.info()

In [ ]:
# Veja que os dados foram reduzidos de 265698  para 262960
# qual a procentagem de dados que foram mantidos ?
print(round(262960 / 265698 * 100,2), "Porcentagem dos dados mantidos.")

98.97 Porcentagem dos dados mantidos.


<pUsar o comando DROPNA pode prejudicar severamente as informações. Neste caso, vemos que somente 1.03% dos dados foram perdidos, o que não deve prejudicar a análise deles</p>

<h2>1. Crimes mais comuns de  Chicago</h2>
<h3>Top 5 Crimes em Chicago em 2018</h3>

In [ ]:
# Set the style of the plot first
plt.style.use('tableau-colorblind10')  ## usando um dos estilos de plot apresentando anteriormente

# ordenando o tipo de crime (Primary Type) em ordem decrescente do total de ocorrencias (value_counts)
top_5_crimes = df['Primary Type'].value_counts().sort_values(ascending=False).head()
# cria um dataset temporário para ser plotado
temp = df.groupby('Primary Type', as_index=False).agg({"ID": "count"})
temp = temp.sort_values(by=['ID'], ascending=False).head()
temp = temp.sort_values(by='ID', ascending=True)
sns.barplot(x='ID', y='Primary Type', data=temp, palette="Blues_d")

# alterando titulo e aparencia do gráfico
plt.title("Top 5 Crimes em Chicago", fontdict = {'fontsize': 30, 'fontname':'Arial', 'color': '#000000'})
plt.xlabel("\nNúmero de Crimes", fontdict = {'fontsize': 15})
plt.ylabel("")
plt.xticks(rotation=90)
plt.show()


<h4>Os dados de date e hora não estão em formato para se trabalhar com Python. Eles usamo AM e PM para diferenciar horas, e outras caracteristicas. Vamos usar um função apra reescrever estes dados</h4>

<h4>Convertendo data e hora para um formato para python </h4>

In [ ]:
# Fun~ção de conversão de data e hora
def time_convert(date_time):
    s1 = date_time[:11]
    s2 = date_time[11:]

    month = s1[:2]
    date = s1[3:5]
    year = s1[6:10]

    hr = s2[:2]
    mins = s2[3:5]
    sec = s2[6:8]
    time_frame = s2[9:]
    if(time_frame == 'PM'):
        if (int(hr) != 12):
            hr = str(int(hr) + 12)
    else:
        if(int(hr) == 12):
            hr = '00'

    final_date = datetime(int(year), int(month), int(date), int(hr), int(mins), int(sec))
    return final_date

In [ ]:
# Aplicando a função acima para todo a coluna Date do dataframe e salvando nela mesma
df['Date'] = df['Date'].apply(time_convert)

<h4>Criando uma coluna MES para melhorar a apresentação dos datos</h4>

In [ ]:
def month(x):
    return x.strftime("%B")
df['Month'] = df['Date'].apply(month)

In [ ]:
#!sudo dpkg-reconfigure locales
#tentativa de instalar locale para pt-br, para que os meses fossem em portugues

In [ ]:
# Frequência dos crimes que mais ocorreram em chicago em 2018
# vamos inicialmente configurar o Python para que trabalhe com datas em portugues, para gerarmos os meses
#from datetime import datetime
#import locale

# Defina a localização para o português do Brasil ('pt_BR')
#locale.setlocale(locale.LC_TIME, 'pt_BR.UTF-8 UTF-8#')

theft_dict ={} # dictionary
battery_dict = {}
crim_dam = {}
assault = {}
dec_prac = {}

months = df["Month"].unique()
for month in months :
    theft_dict[month]=0
    battery_dict[month]=0
    crim_dam[month]=0
    assault[month]=0
    dec_prac[month]=0
# vamos fazer apenas para os 5 mais comuns
for elem in df[df["Primary Type"]=="Roubo"]["Month"]:
    if elem in theft_dict.keys():
        theft_dict[elem] += 1

for elem in df[df["Primary Type"]=="Agressão"]["Month"]:
    if elem in battery_dict.keys():
        battery_dict[elem] += 1

for elem in df[df["Primary Type"]=="Invasão"]["Month"]:
    if elem in crim_dam.keys():
        crim_dam[elem] += 1

for elem in df[df["Primary Type"]=="Assalto"]["Month"]:
    if elem in assault.keys():
        assault[elem] += 1

for elem in df[df["Primary Type"]=="Fraude"]["Month"]:
    if elem in dec_prac.keys():
        dec_prac[elem] += 1



months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
#months = ['Janeiro', 'Fevereiro', 'Março', 'Abril', 'Maio', 'Junho', 'Julho', 'Agosto', 'Setembro', 'Outubro', 'Novembro', 'Dezembro']

theft_list = [(k,theft_dict[k]) for k in months]
battery_list = [(k,battery_dict[k]) for k in months]
crim_dam_list = [(k,crim_dam[k]) for k in months]
assault_list = [(k,assault[k]) for k in months]
dec_prac_list = [(k,dec_prac[k]) for k in months]

In [ ]:
# verificando quantos crimes de Roubos ocorreram por mes (theft_list)

In [ ]:
# Plotting the graphs

plt.style.use('seaborn-dark')
#sns.set(rc={'axes.facecolor':'A0D7E6', 'figure.facecolor':'ffffff'})
fig, ax = plt.subplots(figsize=(12,7))

ax.spines["top"].set_visible(False)
ax.spines["bottom"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.spines["left"].set_visible(False)

# Setting the ticks only on the bottom and the left of the graph
ax.get_xaxis().tick_bottom()
ax.get_yaxis().tick_left()

plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

plt.ylim(500, 6500)

x = [z[0] for z in theft_list]
# print(x)
y = [z[1] for z in theft_list]
ax.plot(x,y, color="black")
ax.lines[0].set_linestyle("--")

x = [z[0] for z in battery_list]
y = [z[1] for z in battery_list]
ax.plot(x,y, color="red")
ax.lines[1].set_linestyle("--")

x = [z[0] for z in crim_dam_list]
y = [z[1] for z in crim_dam_list]
ax.plot(x,y, color="blue")
ax.lines[2].set_linestyle("--")

x = [z[0] for z in assault_list]
y = [z[1] for z in assault_list]
ax.plot(x,y, color="orange")
ax.lines[3].set_linestyle("--")

# x,y = zip(*dec_prac.items())
x = [z[0] for z in dec_prac_list]
y = [z[1] for z in dec_prac_list]
ax.plot(x,y, color="green")
ax.lines[4].set_linestyle("--")


for tick in ax.get_xticklabels():
    tick.set_rotation(90)

plt.text(10,5400,"Roubo",fontsize=18)
plt.text(10,4000,"Agressão",fontsize=18,color="red")
plt.text(10,2400,"Invasão",fontsize=18,color="blue")
plt.text(10,1700,"Assalto",fontsize=18,color="orange")
plt.text(10,600,"Deceptive\nPractice",fontsize=18,color="green")

ax.set_title("Frequencia de ocorrencia dos Top 5 Crimes\n",fontname="monospace", fontsize=20)
ax.set_xlabel("Mês", fontsize=18)
ax.set_ylabel("Número de  Crimes\n", fontsize=16)

plt.show()

In [ ]:
# Dataframes para cada crime, para utilizarmos depois

theft_df = df[df['Primary Type']=='Roubo']
battery_df = df[df['Primary Type']=='Agressão']
crim_dam_df = df[df['Primary Type']=='Invasão']
assault_df = df[df['Primary Type']=='Assalto']
dec_prac_df = df[df['Primary Type']=='Fraude']


<h2>2.Prisões em  Chicago</h2>


In [ ]:
# Arrest são as prisões. O comando abaixo conta os valores. Como só temos True ou False, retorna algo como abaixo
l = df["Arrest"].value_counts()
l

In [ ]:
false = l[0]
true = l[1]

arrest = pd.DataFrame({'Status':['Not Arrested','Arrested'],'Value':list(l)})
print("Porcentagem  de não prisões efetuadas :",false/(false+true)*100,'!')

#Caraca! A change de não ser preso em 2018 era de  80%

<h3>Prisões por meses</h3>

In [ ]:
# verde são prisões efetuadas. Vamos ver por meses quais as quantidades de prisões efetuadas e não efetuadas
plt.style.use('bmh')

fig, ax = plt.subplots(figsize=(10, 5))
ax = sns.countplot(x="Month",
                   hue='Arrest',
                   data=df[['Month','Arrest']],  # usando apenas as colunas Month (criada por nós) e a Arrest , que é False ou True
                   palette=['Red', 'Green'])
months = ['January','February','March','April','May','June','July','August','September','October','November','December']

ax.set( xlabel='Mês', ylabel='Número de Prisões', xticklabels=months)
plt.title('Prisões realizadas em 2018', fontdict={'fontsize': 20, 'color': 'black'}, weight="bold")
plt.show()

<h3>Crimes e prisões ou não prisões</h3>

In [ ]:
# Agrupando (olha o goup by ai...) por prisão (Arrest) e por crimes (Primary Type)
#arrest_crime = df.groupby(['Primary Type', 'Arrest'])['ID'].count()
arrest_crime = df.groupby(['Primary Type', 'Arrest']).agg({'Arrest':"count"})
arrest_crime.columns = ["Count"]
# arrest_crime
arrest_crime

In [ ]:
import math

the_crimes=['Roubo', 'Agressão', 'Invasão', 'Assalto', 'Fraude', 'Violação de Armas', 'Narcóticos', 'Homicídio', 'Roubo Residencial', 'Interferência com Oficial Público']

#the_crimes=['Roubo', 'Agressão', 'Invasão', 'Assalto', 'Fraude']

# the_crimes has only the crimes we want to analyze now

# create the dataframe arrest
arrest = pd.DataFrame(columns=['Crime','True','False'])


In [ ]:

# Criando nosso próprio dataset
for i,crime in enumerate(the_crimes):

    subset = df[df['Primary Type']==crime]
    true = subset['Arrest'].value_counts().sort_index()[1]
    false = subset['Arrest'].value_counts().sort_index()[0]
    temp_df = pd.DataFrame({'Crime':[crime], 'True':[true], 'False':[false]})

    arrest = arrest.append(temp_df)

    arrest['True_per']=arrest['True']/(arrest['True']+arrest['False'])*100
    arrest['False_per']=arrest['False']/(arrest['True']+arrest['False'])*100

##usando lib Bokeh (https://docs.bokeh.org/en/latest/)

In [ ]:
from bokeh.io import output_file
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, CDSView, GroupFilter, HoverTool
from bokeh.io import output_notebook


In [ ]:

# arrest is our dataset

# plot the dataset
def style(p):
        # Title
        p.title.align = 'center'
        p.title.text_font_size = '20pt'
        p.title.text_font = 'serif'

        # Axis titles
        p.xaxis.axis_label_text_font_size = '14pt'
        p.xaxis.axis_label_text_font_style = 'bold'
        p.yaxis.axis_label_text_font_size = '14pt'
        p.yaxis.axis_label_text_font_style = 'bold'

        # Tick labels
        p.xaxis.major_label_text_font_size = '12pt'
        p.yaxis.major_label_text_font_size = '12pt'

        return p

#status = ['Arrested', 'Not Arrested']
status = ['Arrested','Not Arrested']
arrest_cds = ColumnDataSource(arrest)
crimes = arrest_cds.data['Crime'].tolist()
fig2 = figure(x_range=crimes, width = 700, height = 700, title = 'Prisões vs Crime ',     x_axis_label = 'Prisões', y_axis_label = 'Número de Crimes')
fig2.vbar_stack(stackers = ['True','False'], x='Crime',width=0.25,  source=arrest_cds,color=['green','red'])
fig2.xaxis.major_label_orientation = math.pi/2

tooltips = [('Crime','@Crime'),('Arrested Percentage','@True_per'), ('Not Arrested Percentage', '@False_per')]
hover = HoverTool(tooltips=tooltips, mode='vline')

fig2.add_tools(hover)

# fig2=style(fig2)

output_notebook()
show(fig2)

<h2>3. Crime vs Time</h2>
<h3>Distribuição dos crimes ao longo do ano</h3>

In [ ]:
# Set plot style
plt.style.use('ggplot')
sns.set_context('notebook')

# Code to plot
sns.countplot(y='Month', data=df, palette=["#DF0D0D"], order=['January', 'February', 'March', 'April', 'May', 'June', "July", 'August', 'September', 'October', 'November', 'December'], alpha=0.5)

# Aesthetic appeal of the plot
plt.title("Crimes durante o verão !", fontdict={'fontsize': 25, 'color': '#DF0D0D', 'fontname':'Agency FB'}, weight="bold")
plt.ylabel("Mês\n", fontdict={'fontsize': 20}, weight="bold", color="#833636")
plt.xlabel("\nNúmero de  Crimes", fontdict={'fontsize': 20}, weight="bold", color="#833636")

plt.xticks(fontsize=15,color='black')
plt.yticks(fontsize=15, color='black')
plt.show()

<p><bold>Acontecem mais crimes de Maio a Agosto, que é o verão em Chicago, o que é uma importante informação para prevenção!</bold></p>

<h4>Da mesma forma que fizemos com Meses, vamos criar uma coluna nova de horas [24 hour format]</h4>

In [ ]:
def hour(x):
    return x.strftime("%H")
df['Hour_Day'] = df['Date'].apply(hour)

<h3>qual o horário menos seguro ?</h3>

In [ ]:
# Set plot style
plt.style.use('seaborn-dark')
sns.set_context('paper')

df_horas = df.sort_values(by='Hour_Day')


# Write code to plot
fig, ax = plt.subplots(figsize=(10, 5))
sns.countplot(x='Hour_Day', data=df_horas, palette="viridis")

# Aesthetic appeal
plt.title("Horário Menos Seguro em  Chicago em 2018", fontdict={'fontsize': 25, 'color': '#bb0e14','fontname':'Agency FB'}, weight="bold")
plt.xlabel("\nHora", fontdict={'fontsize': 15}, weight='bold')
plt.ylabel("Número de Crimes\n", fontdict={'fontsize': 15}, weight="bold")

# Add Text to the plot
plt.text(2, 7000, 'Menor taxa de criminalidade Crime Rate', fontdict={'fontsize': 14, 'color':"blue" }, weight='bold')

plt.show()

<p>Que estranho, não é? A Taxa de criminalidade cai da meia-noite (00 horas) para as 05 horas.<br>
    <strong>Além do frio forte, criminosos também dormem à noite!</strong>
</p>

<h2>3. Crime vs Localização</h2>
<h3>Um mapa simples de CHicago Usando as coordenadas</h3>

In [ ]:
#
#sns.lmplot('X Coordinate','Y Coordinate', data=df, fit_reg=False, hue="District", palette='colorblind', height=5,scatter_kws={"marker": "+","s": 10})
#sns.lmplot('X Coordinate', 'Y Coordinate', df, fit_reg=False, hue="District", palette='colorblind', height=5, scatter_kws={"marker": "+", "s": 10})
sns.lmplot(x='X Coordinate', y='Y Coordinate', data=df, fit_reg=False, hue="District", palette='colorblind', height=5, scatter_kws={"marker": "+", "s": 10})


ax = plt.gca()
ax.set_title("Mapa das regiões de Chicago\n", fontdict={'fontsize': 15}, weight="bold")
plt.show()

<h3>Ocorrencias mais comuns por distrito</h3>

In [ ]:
# location attributes = ['Location Description','Beat','District', 'Ward', 'Community Area','X Coordinate','Y Coordinate', 'Latitude', 'Longitude','Location']
#%matplotlib inline
top = df.groupby(['District', 'Primary Type']).size().reset_index(name='counts').groupby('District').apply(lambda x: x.sort_values('counts',ascending=False).head(5))
#print(topk)

# factor plot to make multiple plots
g = sns.catplot(x="Primary Type", y='counts', col="District", col_wrap=3, data=top, kind='bar')
for ax in g.axes:
    plt.setp(ax.get_xticklabels(), visible=True, rotation=45, ha='right')
plt.subplots_adjust(hspace=0.4)

In [ ]:
ax.get_xticklabels()

[Text(0, 0, 'Roubo'),
 Text(1, 0, 'Fraude'),
 Text(2, 0, 'Agressão'),
 Text(3, 0, 'Invasão'),
 Text(4, 0, 'Assalto'),
 Text(5, 0, 'Outra Ofensa'),
 Text(6, 0, 'Narcóticos'),
 Text(7, 0, 'Roubo Residencial'),
 Text(8, 0, 'Roubo de Veículo')]

In [ ]:
## Crime data distribution of gang based criminal activities
import folium

dp=df[df['Primary Type']=='Fraude']
battery = df[df['Primary Type']=='Agressão']
cd = df[df['Primary Type']=='Invasão']
mtheft = df[df['Primary Type']=='Roubo de Veículo']
theft = df[df['Primary Type']=='Roubo']
narcotics= df[df['Primary Type']=='Narcóticos']
merge = cd.append(battery)
merge = merge.append(mtheft)
#merge = merge.append(cd)
merge = merge.append(dp)
merge = merge.append(theft)
merge = merge.append(narcotics)
chicago_map = folium.Map(location=[41.864073,-87.706819],
                        zoom_start=11,
                        tiles="openstreetmap")
locations = merge.groupby('Primary Type').nth(7)


new_locations = locations.loc[:, ['Latitude', 'Longitude', 'Location Description', 'Arrest']]


print(new_locations.head(6))


popup_text = """Community Index : {}<br
                Arrest : {}<br>
                Location Description : {}<br>"""


for i in range(len(new_locations)):
    lat = new_locations.iloc[i][0]
    long = new_locations.iloc[i][1]
    popup_text = """Community Index : {}<br>
                Arrest : {}<br>
                Location Description : {}<br>"""
    popup_text = popup_text.format(new_locations.index[i],
                               new_locations.iloc[i][-1],
                               new_locations.iloc[i][-2]
                               )
    folium.CircleMarker(location = [lat, long], popup= popup_text, fill = True).add_to(chicago_map)


#chicago_map.save('map.html')
display(chicago_map)

#chicago_map._build_map()


In [ ]:
a=''
a=input("Entre com o tipo de crime para visualizar onde mais ocorrem (obs. Digital igual aos crimes em portugues (ex. Roubo, Homicídio, etc)) ")

var=df[df['Primary Type']==a]
unique_locations = var['Location'].value_counts()
print("%%%%%%%%%%%%%%%%%%%%%%%%%%% ",len(unique_locations))
#print("%%%%%%%%%%%%%%%%%%%%%%%%%%% ",unique_locations)


unique_locations.index


CR_index = pd.DataFrame({"Raw_String" : unique_locations.index, "ValueCount":unique_locations})
CR_index.index = range(len(unique_locations))
print(CR_index.head())
print("valuecount",len(CR_index['ValueCount']))
def Location_extractor(Raw_Str):
    preProcess = Raw_Str[1:-1].split(',')
    lat =  float(preProcess[0])
    long = float(preProcess[1])
    return (lat, long)


CR_index['LocationCoord'] = CR_index['Raw_String'].apply(Location_extractor)


CR_index  = CR_index.drop(columns=['Raw_String'], axis = 1)


#chicago_map._build_map()
#%%time

chicago_map_crime = folium.Map(location=[41.895140898, -87.624255632],
                        zoom_start=15,
                        tiles="openstreetmap")

for i in range(500):
    lat = CR_index['LocationCoord'].iloc[i][0]
    long = CR_index['LocationCoord'].iloc[i][1]
    radius = CR_index['ValueCount'].iloc[i] / 50

    if CR_index['ValueCount'].iloc[i] > 60:
        color = "#FF4500"
    else:
        color = "#008080"

    popup_text = """Latitude : {}<br>
                Longitude : {}<br>
                Criminal Incidents : {}<br>"""
    popup_text = popup_text.format(lat,
                               long,
                               CR_index['ValueCount'].iloc[i]
                               )
    folium.CircleMarker(location = [lat, long], popup= popup_text,radius = radius, color = color, fill = True).add_to(chicago_map_crime)

#chicago_map_crime.save('crime.html')

display(chicago_map_crime)


<h3>Usando outro g´rafico para ver a distribuição por determinado tipo de crime</h3>

In [ ]:
# let's take in the crime as an input as always
c = input("Entre o tipo de crime (Roubo, Homicídio, etc)>> ")
crime_df = df[df['Primary Type']==c]

sns.jointplot(x=crime_df['X Coordinate'].values, y=crime_df['Y Coordinate'].values, height=7, kind='hex')

plt.xlabel("Latitude", fontdict={'fontsize': 12}, weight='bold')
plt.ylabel("Longitude", fontdict={'fontsize': 12}, weight="bold")
plt.show()